In [29]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 19: Medicine for Rudolph ---</h2><p>Rudolph the Red-Nosed Reindeer is sick!  His nose isn't shining very brightly, and he needs medicine.</p>
<p>Red-Nosed Reindeer biology isn't similar to regular reindeer biology; Rudolph is going to need custom-made medicine.  Unfortunately, Red-Nosed Reindeer chemistry isn't similar to regular reindeer chemistry, either.</p>
<p>The North Pole is equipped with a Red-Nosed Reindeer nuclear fusion/fission plant, capable of constructing any Red-Nosed Reindeer molecule you need.  It works by starting with some input molecule and then doing a series of <em>replacements</em>, one per step, until it has the right molecule.</p>
<p>However, the machine has to be calibrated before it can be used.  Calibration involves determining the number of molecules that can be generated in one step from a given starting point.</p>
<p>For example, imagine a simpler machine that supports only the following replacements:</p>
<pre><code>H =&gt; HO
H =&gt; OH
O =&gt; HH
</code></pre>
<p>Given the replacements above and starting with <code>HOH</code>, the following molecules could be generated:</p>
<ul>
<li><code>HOOH</code> (via <code>H =&gt; HO</code> on the first <code>H</code>).</li>
<li><code>HOHO</code> (via <code>H =&gt; HO</code> on the second <code>H</code>).</li>
<li><code>OHOH</code> (via <code>H =&gt; OH</code> on the first <code>H</code>).</li>
<li><code>HOOH</code> (via <code>H =&gt; OH</code> on the second <code>H</code>).</li>
<li><code>HHHH</code> (via <code>O =&gt; HH</code>).</li>
</ul>
<p>So, in the example above, there are <code>4</code> <em>distinct</em> molecules (not five, because <code>HOOH</code> appears twice) after one replacement from <code>HOH</code>. Santa's favorite molecule, <code>HOHOHO</code>, can become <code>7</code> <em>distinct</em> molecules (over nine replacements: six from <code>H</code>, and three from <code>O</code>).</p>
<p>The machine replaces without regard for the surrounding characters.  For example, given the string <code>H2O</code>, the transition <code>H =&gt; OO</code> would result in <code>OO2O</code>.</p>
<p>Your puzzle input describes all of the possible replacements and, at the bottom, the medicine molecule for which you need to calibrate the machine.  <em>How many distinct molecules can be created</em> after all the different ways you can do one replacement on the medicine molecule?</p>
</article>


In [30]:
from collections import deque
from functools import cache
from typing import Generator
from tabulate import tabulate


replacements = """
H => HO
H => OH
O => HH
"""

medicine = "HOH"
end = "HOHOHO"


class RedNosedReindeerNuclearFusionFissionPlant:
    def __init__(self, replacements: str) -> None:
        self.replacements = defaultdict(list)

        for l in replacements.strip().splitlines():
            fr, to = l.split(" => ")
            self.replacements[fr].append(tuple(re.findall(r"[A-Z]{1}[a-z]*", to)))

    def replace(self, m: tuple[str, ...] | str) -> Generator[tuple[str, ...]]:
        if isinstance(m, str):
            m = tuple(re.findall(r"[A-Z]{1}[a-z]*", m))

        for i in range(len(m)):
            for reps in self.replacements[m[i]]:
                yield m[:i] + reps + m[i + 1 :]

    def minimumsteps(self, end: str) -> int:
        # too slow
        end = tuple(re.findall(r"[A-Z]{1}[a-z]*", end))

        queue = {("e",)}
        steps = 0

        while queue:
            steps += 1
            new_queue = set()
            for m in queue:
                new_queue |= set(self.replace(m))

            if end in new_queue:
                return steps

            queue = new_queue

        return -1

    def minimumsteps_dfs(self, end: str) -> int:
        # Craches computer
        @cache
        def dfs(m: tuple[str, ...], steps: int) -> int:
            if len(m) > len(end):
                return inf
            if m in seen:
                return inf
            seen.add(m)

            if m == end:
                return steps

            minimum = inf
            for i, e in enumerate(m):
                for r in self.replacements[e]:
                    l = dfs(m[:i] + r + m[i + 1 :], steps + 1)
                    if l < minimum:
                        minimum = l
            return minimum

        seen = set()
        end = tuple(re.findall(r"[A-Z]{1}[a-z]*", end))
        return dfs(("e",), 0)

    def __repr__(self) -> str:
        return tabulate(self.replacements, headers=self.replacements.keys())


r = RedNosedReindeerNuclearFusionFissionPlant(replacements)
print("Distinct molecules that can be created", len(set(r.replace(medicine))))

Distinct molecules that can be created 4


In [31]:
puzzle_replacements = """
Al => ThF
Al => ThRnFAr
B => BCa
B => TiB
B => TiRnFAr
Ca => CaCa
Ca => PB
Ca => PRnFAr
Ca => SiRnFYFAr
Ca => SiRnMgAr
Ca => SiTh
F => CaF
F => PMg
F => SiAl
H => CRnAlAr
H => CRnFYFYFAr
H => CRnFYMgAr
H => CRnMgYFAr
H => HCa
H => NRnFYFAr
H => NRnMgAr
H => NTh
H => OB
H => ORnFAr
Mg => BF
Mg => TiMg
N => CRnFAr
N => HSi
O => CRnFYFAr
O => CRnMgAr
O => HP
O => NRnFAr
O => OTi
P => CaP
P => PTi
P => SiRnFAr
Si => CaSi
Th => ThCa
Ti => BP
Ti => TiTi
e => HF
e => NAl
e => OMg
"""

medicine = "ORnPBPMgArCaCaCaSiThCaCaSiThCaCaPBSiRnFArRnFArCaCaSiThCaCaSiThCaCaCaCaCaCaSiRnFYFArSiRnMgArCaSiRnPTiTiBFYPBFArSiRnCaSiRnTiRnFArSiAlArPTiBPTiRnCaSiAlArCaPTiTiBPMgYFArPTiRnFArSiRnCaCaFArRnCaFArCaSiRnSiRnMgArFYCaSiRnMgArCaCaSiThPRnFArPBCaSiRnMgArCaCaSiThCaSiRnTiMgArFArSiThSiThCaCaSiRnMgArCaCaSiRnFArTiBPTiRnCaSiAlArCaPTiRnFArPBPBCaCaSiThCaPBSiThPRnFArSiThCaSiThCaSiThCaPTiBSiRnFYFArCaCaPRnFArPBCaCaPBSiRnTiRnFArCaPRnFArSiRnCaCaCaSiThCaRnCaFArYCaSiRnFArBCaCaCaSiThFArPBFArCaSiRnFArRnCaCaCaFArSiRnFArTiRnPMgArF"

rn = RedNosedReindeerNuclearFusionFissionPlant(puzzle_replacements)
print("Distinct molecules that can be created", len(set(rn.replace(medicine))))

Distinct molecules that can be created 576


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>576</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now that the machine is calibrated, you're ready to begin molecule fabrication.</p>
<p>Molecule fabrication always begins with just a single <span title="It's a Red-Nosed Reindeer electron.">electron</span>, <code>e</code>, and applying replacements one at a time, just like the ones during calibration.</p>
<p>For example, suppose you have the following replacements:</p>
<pre><code>e =&gt; H
e =&gt; O
H =&gt; HO
H =&gt; OH
O =&gt; HH
</code></pre>
<p>If you'd like to make <code>HOH</code>, you start with <code>e</code>, and then make the following replacements:</p>
<ul>
<li><code>e =&gt; O</code> to get <code>O</code></li>
<li><code>O =&gt; HH</code> to get <code>HH</code></li>
<li><code>H =&gt; OH</code> (on the second <code>H</code>) to get <code>HOH</code></li>
</ul>
<p>So, you could make <code>HOH</code> after <em><code>3</code> steps</em>.  Santa's favorite molecule, <code>HOHOHO</code>, can be made in <em><code>6</code> steps</em>.</p>
<p>How long will it take to make the medicine?  Given the available <em>replacements</em> and the <em>medicine molecule</em> in your puzzle input, what is the <em>fewest number of steps</em> to go from <code>e</code> to the medicine molecule?</p>
</article>

</main>


In [32]:
from random import shuffle


replacements = """
e => H
e => O
H => HO
H => OH
O => HH
"""


class Reducer:
    def __init__(self, replacements: str) -> None:
        self.reductions = [
            tuple(reversed(l.split(" => "))) for l in replacements.strip().splitlines()
        ]

    def minimum_steps(self, m: str) -> int:
        while True:
            shuffle(self.reductions)
            current = m
            previous = None
            steps = 0
            while current != previous:
                previous = current
                for fr, to in self.reductions:
                    if to == "e" and len(current) - len(fr) > 0:
                        continue

                    i = current.find(fr)

                    if i < 0:
                        continue
                    steps += 1
                    current = current[:i] + to + current[i + len(fr) :]

                    if current == "e":
                        return steps

    def __repr__(self) -> str:
        return tabulate(self.reductions)


r = Reducer(replacements)
r.minimum_steps("HOHOHO")

6

In [33]:
rf = Reducer(puzzle_replacements)
rf.minimum_steps(medicine)

207

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>207</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="19/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Medicine+for+Rudolph%22+%2D+Day+19+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F19&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Medicine+for+Rudolph%22+%2D+Day+19+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F19';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>


It is a context free grammar: https://en.wikipedia.org/wiki/Context-free_grammar
